# 文档增强与问题生成
为文本chunk生成相关问题，提高检索过程
提取文档、分块、**生成问题**、chunk与问题嵌入、向量存储、语义搜索、响应生成、评估

In [ ]:
import fitz
import numpy as np
import json
from zhipuai import ZhipuAI
import random
from tqdm import tqdm

In [26]:
pdf_path = "AI_Information.en.zh-CN.pdf"
API_KEY = "f532b4bd71324c1ca2fd7e22d4eb41da.4Sfpr8wE6qEVV5BY"
# 设置客户端,处理文本
client = ZhipuAI(api_key = API_KEY) 

# model
llm_model = "glm-4"
embedding_model="embedding-3"

**提取文本识别章节标题**

In [27]:
def extract_text_from_pdf(pdf_path):
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

**提取文本分块**

In [28]:
def chunk_text(text, n, overlap):
    chunks = []  #
    for i in range(0, len(text), n - overlap):
        # 添加从当前索引到索引 + 块大小的文本块
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks

**对每个文本块生成可以通过该文本回答的问题**

In [29]:
def generate_questions(text_chunk, num_questions=5):
    """
    生成可以从给定文本块中回答的相关问题。

    Args:
    text_chunk (str): 要生成问题的文本块。
    num_questions (int): 要生成的问题数量。
    model (str): 用于生成问题的模型。

    Returns:
    List[str]: 生成的问题列表。
    """
    # 定义系统提示
    system_prompt = "你是一个从文本中生成相关问题的专家。能够根据用户提供的文本生成可回答的简洁问题，重点聚焦核心信息和关键概念。"

    # 定义用户提示，包含文本块和要生成的问题数量
    # user_prompt = f"""
    # 根据以下文本，生成 {num_questions} 个不同的问题，这些问题只能通过此文本回答：
    #
    # {text_chunk}
    #
    # 请以编号列表的形式回复问题，且不要添加任何额外文本。
    # """
    user_prompt = f"""
    请根据以下文本内容生成{num_questions}个不同的、仅能通过该文本内容回答的问题：

    {text_chunk}

    请严格按以下格式回复：
    1. 带编号的问题列表
    2. 仅包含问题
    3. 不要添加任何其他内容
    """

    # 使用 API 生成问题
    response = client.chat.completions.create(
        model=llm_model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 从响应中提取并清理问题
    questions_text = response.choices[0].message.content.strip()

    # 使用正则表达式模式匹配提取问题
    # pattern = r'^\d+\.\s*(.*)'
    # return [re.match(pattern, line).group(1) for line in questions_text.split('\n') if line.strip()]
    questions = []
    for line in questions_text.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        # 检查行是否以数字开头
        if line[0].isdigit():
            # 查找第一个非数字和非标点符号的位置
            for i, char in enumerate(line):
                if not (char.isdigit() or char in '.)'):
                    break
            # 提取问题内容（跳过编号和标点）
            question = line[i:].strip()
            if question:
                questions.append(question)
        else:
            # 如果行不以数字开头，可能是没有编号的问题
            questions.append(line)
    
    return questions[:num_questions] 


In [30]:
# 测试生成的问题

print("从 PDF 中提取文本...")
extracted_text = extract_text_from_pdf(pdf_path)

print("分割文本...")
text_chunks = chunk_text(extracted_text, 1000, 200)
print(f"创建了 {len(text_chunks)} 个文本块")

print("处理文本块并生成问题...")

for i, chunk in enumerate(tqdm(text_chunks, desc="处理文本块")):
    questions = generate_questions(chunk, num_questions=5)
    print("Generated Questions:")
    print(questions)
    print("生成问题".center(80, '-'))

    if i == 0:
        break

从 PDF 中提取文本...
分割文本...
创建了 13 个文本块
处理文本块并生成问题...


处理文本块:   0%|          | 0/13 [00:02<?, ?it/s]

Generated Questions:
['人工智能指的是什么能力？', '人工智能研究的正式领域始于哪个世纪？', '1956年的哪个会议被认为是人工智能的发源地？', '监督学习算法是如何进行训练的？', '深度学习使用什么来分析数据？']
--------------------------------------生成问题--------------------------------------


**文本嵌入** 

In [31]:
def create_embeddings(text):
    """
    为给定文本创建嵌入向量，使用指定的 zhipu 模型。

    Args:
    text (str): 要为其创建嵌入向量的输入文本。
    model (str): 用于创建嵌入向量的模型。

    Returns:
    response: 包含嵌入向量的 zhipu API 响应。
    """
    # 使用指定模型为输入文本创建嵌入向量
    response = client.embeddings.create(
        model=embedding_model,
        input=text
    )

    return response  # 返回包含嵌入向量的响应

**定义新的向量存储**

In [44]:
class SimpleVectorStore:
    def __init__(self):
        self.vectors = []  # 存储嵌入向量的列表
        self.texts = []  # 存储对应文本的列表
        self.metadata = []  # 存储元数据的列表

    def add_item(self, text, embedding, metadata=None):
        """
        添加一个文本、其嵌入向量和可选的元数据到存储中。

        Args:
        text (str): 要存储的文本。
        embedding (List[float]): 文本的嵌入向量。
        metadata (dict, optional): 相关的元数据。
        """
        self.texts.append(text)
        self.vectors.append(np.array(embedding))  # 确保嵌入向量是 NumPy 数组
        self.metadata.append(metadata if metadata else {})

    def similarity_search(self, query_embedding, k=5):
        """
        执行相似性搜索，返回与查询嵌入向量最相似的文本。

        Args:
        query_embedding (List[float]): 查询的嵌入向量。
        top_k (int): 返回的最相似文本的数量。

        Returns:
        List[Dict]: 包含最相似文本及其相似度分数的列表。
        """
        if not self.vectors:
            return []
        
        # 转换查询嵌入向量
        query_vector = np.array(query_embedding)

        # 相似度计算
        similarities = []
        for i,vector in enumerate(self.vectors):
            similarity = np.dot(vector, query_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i,similarity))

        # 相似度排序
        similarities.sort(key=lambda x: x[1], reverse=True)  # 按相似度降序排序
        results = []
        for i in range(min(k, len(similarities))):  # 确保不会超出向量数量
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 对应的文本
                "metadata": self.metadata[idx],  # 对应的元数据
                "similarity": score  # 相似度分数
            })

        return results


**使用问题增强处理文档，整合步骤处理文档、生成问题，构建增强向量存储**

In [45]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200, questions_per_chunk=5):
    """
    处理结合了生成问题的文档，结合之前的重叠等方法。

    Args:
    pdf_path (str): PDF 文件的路径。
    chunk_size (int): 每个文本块的字符大小。
    chunk_overlap (int): 块之间的重叠字符数。
    questions_per_chunk (int): 每个块生成的问题数量。

    Returns:
    Tuple[List[str], SimpleVectorStore]: 文本块列表和向量存储。
    """
    print("从 PDF 中提取文本...")
    extracted_text = extract_text_from_pdf(pdf_path)

    print("分割文本...")
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"创建了 {len(text_chunks)} 个文本块")

    # 初始化向量存储
    # print("初始化向量存储...")
    vector_store = SimpleVectorStore()

    print("处理文本块并生成问题...")
    for i, chunk in enumerate(tqdm(text_chunks, desc="处理文本块")):
        # 为文本块本身创建嵌入
        chunk_embedding_response = create_embeddings(chunk)
        chunk_embedding = chunk_embedding_response.data[0].embedding

        # 将文本块添加到向量存储中
        vector_store.add_item(
            text=chunk,
            embedding=chunk_embedding,
            metadata={"type": "chunk", "index": i}
        )

        # 为该文本块生成问题
        questions = generate_questions(chunk, num_questions=questions_per_chunk)

        # 为每个问题创建嵌入并添加到向量存储中
        for j, question in enumerate(questions):
            question_embedding_response = create_embeddings(question)
            question_embedding = question_embedding_response.data[0].embedding

            # 将问题添加到向量存储中
            vector_store.add_item(
                text=question,
                embedding=question_embedding,
                metadata={"type": "question", "chunk_index": i, "original_chunk": chunk}
            )

    return text_chunks, vector_store


**文档处理**

In [46]:
# 处理文档（提取文本、创建块、生成问题、构建向量存储）
text_chunks, vector_store = process_document(
    pdf_path,
    chunk_size=1000,  # 每个文本块的字符大小为1000
    chunk_overlap=200,  # 块之间的重叠字符数为200
    questions_per_chunk=3  # 每个块生成3个问题
)

# 打印向量存储中的项目数量
print(f"向量存储包含 {len(vector_store.texts)} 个项目")  # 13*4=52，一个块由1个chunk,3个question组成


从 PDF 中提取文本...
分割文本...
创建了 13 个文本块
处理文本块并生成问题...


处理文本块: 100%|██████████| 13/13 [00:53<00:00,  4.10s/it]

向量存储包含 52 个项目


**增强向量存储的语音检索增强生成**

In [47]:
def semantic_search(query, vector_store, k=5):
    """
    使用查询和向量存储执行语义搜索。

    Args：
    query (str): 搜索查询。
    vector_store (SimpleVectorStore): 要搜索的向量存储。
    k (int): 返回的结果数量。

    Returns：
    List[Dict]: 最相关的前 k 个结果列表，每个结果包含文本和元数据信息。
    """
    # 为查询创建嵌入
    query_embedding_response = create_embeddings(query)
    query_embedding = query_embedding_response.data[0].embedding

    # 搜索向量存储
    results = vector_store.similarity_search(query_embedding, k=k)

    return results

**向量存储中检索问题、生成**

In [49]:
# 测试语义搜索
with open("val.json", mode="r", encoding="utf-8") as f:
    data = json.load(f)

# 随机测试
random_index = random.randint(0, len(data) - 1)
query = data[2]["question"]

# 执行语义搜索
search_results = semantic_search(query, vector_store, k=5)

print("Query:", query)  # 打印查询内容
print("\nSearch Results:")  # 打印搜索结果标题

chunk_results = []  # 文档块的结果
question_results = []  # 问题的结果

for result in search_results:
    if result["metadata"]["type"] == "chunk":  # 如果结果是文档块类型
        chunk_results.append(result)
    else:  # 如果结果是问题类型
        question_results.append(result)

# 打印文档块结果
print("\nRelevant Document Chunks:")  # 打印相关文档块标题
for i, result in enumerate(chunk_results):
    print(f"Context {i + 1} (similarity: {result['similarity']:.4f}):")  # 打印每个文档块的相似度分数
    print(result["text"][:300] + "...")  # 打印文档块的前300个字符
    print("=====================================")  # 分隔符

# 打印匹配的问题
print("\nMatched Questions:")  # 打印匹配问题标题
for i, result in enumerate(question_results):
    print(f"Question {i + 1} (similarity: {result['similarity']:.4f}):")  # 打印每个问题的相似度分数
    print(result["text"])  # 打印问题内容
    chunk_idx = result["metadata"]["chunk_index"]  # 获取问题所属的文档块索引
    print(f"From chunk {chunk_idx}")  # 打印问题来源的文档块索引
    print("=====================================")  # 分隔符



Query: 与人工智能驱动的人脸识别相关的伦理问题有哪些？

Search Results:

Relevant Document Chunks:

Matched Questions:
Question 1 (similarity: 0.6627):
在人工智能安全与保障方面，研究的关键点是什么？
From chunk 9
Question 2 (similarity: 0.6499):
在负责任地开发和部署人工智能中，需要遵守哪些原则？
From chunk 12
Question 3 (similarity: 0.6438):
在人工智能发展中，为什么需要建立指导方针和道德框架？
From chunk 3
Question 4 (similarity: 0.6414):
什么是以人为本的人工智能方法？
From chunk 12
Question 5 (similarity: 0.6215):
人工智能在人力资源领域有哪些具体应用？
From chunk 5


**结合文档块、问题准备回答的上下文**

In [50]:
def prepare_context_for_answering(search_results):
    """
    从语义搜索中准备回答问题的上下文。

    Args:
    search_results (List[Dict]): 语义搜索的结果。

    Returns:
    str: 准备好的上下文字符串。
    """
    # 结果中独特文档块
    chunk_indices = set()
    context_chunks = []

    # 添加直接匹配的文档块
    for result in search_results:
        if result["metadata"]["type"] == "chunk":
            chunk_indices.add(result["metadata"]["index"])
            context_chunks.append(f"Chunk {result['metadata']['index']}:\n{result['text']}")

    # 添加问题引用的文档块
    for result in search_results:
        if result["metadata"]["type"] == "question":
            chunk_idx = result["metadata"]["chunk_index"]
            if chunk_idx not in chunk_indices:
                chunk_indices.add(chunk_idx)
                context_chunks.append(
                    f"Chunk {chunk_idx} (referenced by question '{result['text']}'):\n{result['metadata']['original_chunk']}")

    # 合并上下文块
    full_context = "\n\n".join(context_chunks)
    return full_context

**根据检索上下文生成回答**

In [51]:
def generate_response(query, context):
    # 定义系统提示，指导zhipu严格基于给定的上下文进行回答。
    # 如果无法从上下文回复，应该说不知道而不是瞎说
    system_prompt = "你是一个AI助手，严格根据给定的上下文进行回答。如果无法直接从提供的上下文中得出答案，请回复：'我没有足够的信息来回答这个问题。'"

    # 使用上下文和问题定义用户提示
    user_prompt = f"""
        上下文内容:
        {context}

        问题: {query}

        请仅根据上述上下文回答问题, 并保持简明扼要。
    """

    response = client.chat.completions.create(
        model=llm_model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content  # 返回生成的回答

In [52]:
# 准备上下文
context = prepare_context_for_answering(search_results)

# 生成回答
response = generate_response(query, context)

print("\nQuery:", query)
print("\nResponse:")
print(response)



Query: 与人工智能驱动的人脸识别相关的伦理问题有哪些？

Response:
与人工智能驱动的人脸识别相关的伦理问题包括隐私权、数据保护、公平性和透明度。具体而言，人脸识别技术可能会未经个人同意收集和利用个人生物识别信息，这侵犯了隐私权；此外，若数据管理不当，可能会导致敏感信息泄露，引发数据保护问题。还有，人脸识别系统可能存在算法偏见，对不同人群的识别准确性和公平性存疑，缺乏透明度也会使得系统的决策过程难以接受公众监督。


**eval**

In [53]:
def evaluate_response(query, response, reference_answer):
    """
    对AI生成的回答进行评估，将其与参考答案进行对比。

    Args:
    query (str): 用户的问题。
    response (str): AI生成的回答。
    reference_answer (str): 参考/理想答案。
    model (str): 用于评估的模型。

    Returns:
    str: 评估反馈。
    """
    # 定义评估系统的系统提示
    evaluate_system_prompt = """您是一个智能评估系统，负责评估AI回答的质量。
    请将AI助手的回答与真实/参考答案进行对比，基于以下几点进行评估：
        1. 事实正确性 - 回答是否包含准确信息？
        2. 完整性 - 是否涵盖参考内容的所有重要方面？
        3. 相关性 - 是否直接针对问题作出回应？

        请分配0到1之间的评分：
        - 1.0：内容与含义完全匹配
        - 0.8：非常好，仅有少量遗漏/差异
        - 0.6：良好，涵盖主要要点但遗漏部分细节
        - 0.4：部分正确答案但存在显著遗漏
        - 0.2：仅包含少量相关信息
        - 0.0：错误或无关信息

    请提供评分并附理由说明。
    """

    # 创建评估提示
    # 包含用户问题、AI回答、参考答案以及要求评估的内容。
    evaluation_prompt = f"""
        用户问题: {query}

        AI回答:
        {response}

        参考答案:
        {reference_answer}

        请根据参考答案评估AI的回答。
    """

    # 生成评估结果
    # 使用指定的模型生成评估结果。
    eval_response = client.chat.completions.create(
        model=llm_model,
        temperature=0,
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluation_prompt}
        ]
    )

    # 返回评估内容
    return eval_response.choices[0].message.content

reference_answer = data[0]['ideal_answer']

evaluation = evaluate_response(query, response, reference_answer)

print("\nEvaluation:")
print(evaluation)


Evaluation:
根据提供的参考答案，以下是对AI回答的评估：

1. 事实正确性：AI回答提到了人脸识别技术涉及的主要伦理问题，如隐私权、数据保护、公平性和透明度，这些都是当前关于人脸识别伦理讨论中的关键议题。参考答案虽然提到了可解释人工智能（XAI），但并未直接列举人脸识别的伦理问题。因此，AI的回答在事实正确性方面是符合要求的。
   评分：0.8

2. 完整性：AI回答较为全面地概述了人脸识别技术的伦理问题，包括多个重要方面。虽然它没有提到“可解释人工智能”（XAI）这一特定术语，但其所提及的透明度和公平性实际上与XAI的概念有所重叠。因此，在完整性方面，AI回答做得相对较好。
   评分：0.7

3. 相关性：AI的回答直接针对了问题，即人脸识别相关的伦理问题，与问题的主题紧密相关。
   评分：1.0

综合评分：根据以上三个标准，综合评分为：(0.8 + 0.7 + 1.0) / 3 = 0.833

理由说明：
- AI的回答在事实正确性方面做得很好，涵盖了人脸识别技术的主要伦理问题。
- 在完整性方面，虽然AI回答没有涵盖参考答案中提到的“可解释人工智能”这一术语，但其内容仍然包括了与该术语相关的概念，因此只轻微扣分。
- 在相关性方面，AI的回答完全符合问题要求，直接回应了问题。

因此，综合考虑，AI的回答质量是较高的。
